In [1]:
import sys
sys.path.insert(0, '../')

# sys.path.remove("/home/users/yixiuz/.local/lib/python3.9/site-packages")
sys.path.append("/home/groups/swl1/yixiuz/torch_fid/lib/python3.9/site-packages")
sys.path.append("/home/groups/swl1/yixiuz/torch_fid/bin")

In [2]:
from importlib import reload

In [6]:
# reload(dataset_utils)
# reload(datasets)
# reload(countdown)
reload(piano)
# reload(losses_utils)
# reload(losses)
reload(model_utils)
reload(models)

<module 'lib.models.models' from '../lib/models/models.py'>

In [7]:
import torch
import torch.nn as nn
import ml_collections
import yaml
import lib.utils.bookkeeping as bookkeeping
from pathlib import Path
import torch.utils.tensorboard as tensorboard
from tqdm import tqdm
import sys
import signal
import argparse


import lib.models.models as models
import lib.models.model_utils as model_utils
import lib.datasets.datasets as datasets
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
import lib.loggers.loggers as loggers
import lib.loggers.logger_utils as logger_utils

# import config.train.countdown as countdown
# cfg = countdown.get_config()

import config.train.piano_hollow as piano
cfg = piano.get_config()

custom_name = None

print("Training with config", cfg.experiment_name)

preempted_path = Path("null")
if cfg.saving.enable_preemption_recovery:

    preempted_path = bookkeeping.check_for_preempted_run(cfg.save_location,
        cfg.saving.preemption_start_day_YYYYhyphenMMhyphenDD,
        cfg,
        cfg.saving.prepare_to_resume_after_timeout
    )

if preempted_path.as_posix() == "null":
    save_dir, checkpoint_dir, config_dir = \
        bookkeeping.create_experiment_folder(
            cfg.save_location,
            cfg.experiment_name if custom_name is None else custom_name,
            custom_name is None
    )
    bookkeeping.save_config_as_yaml(cfg, config_dir)

    # bookkeeping.save_git_hash(save_dir)

else:
    print("Resuming from preempted run: ", preempted_path)
    save_dir = preempted_path
    checkpoint_dir, config_dir = bookkeeping.create_inner_experiment_folders(save_dir)

writer = bookkeeping.setup_tensorboard(save_dir, 0)

device = torch.device(cfg.device)

model = model_utils.create_model(cfg, device)
print("number of parameters: ", sum([p.numel() for p in model.parameters()]))

dataset = dataset_utils.get_dataset(cfg, device)
dataloader = torch.utils.data.DataLoader(dataset,
    batch_size=cfg.data.batch_size,
    shuffle=cfg.data.shuffle)

loss = losses_utils.get_loss(cfg)

training_step = training_utils.get_train_step(cfg)

optimizer = optimizers_utils.get_optimizer(model.parameters(), cfg)

state = {
    'model': model,
    'optimizer': optimizer,
    'n_iter': 0
}

bookkeeping.setup_preemption(save_dir, checkpoint_dir, state,
    cfg.saving.num_checkpoints_to_keep,
    cfg.saving.prepare_to_resume_after_timeout)


if not preempted_path.as_posix() == 'null':
    state = bookkeeping.resume_training(preempted_path, state, cfg.device)

low_freq_loggers = []
for logger in cfg.saving.low_freq_loggers:
    low_freq_loggers.append(logger_utils.get_logger(logger))

exit_flag = False


while True:
    for minibatch in tqdm(dataloader):

        training_step.step(state, minibatch, loss, writer)

        if state['n_iter'] % cfg.saving.checkpoint_freq == 0 or state['n_iter'] == cfg.training.n_iters-1:
            bookkeeping.save_checkpoint(checkpoint_dir, state,
                cfg.saving.num_checkpoints_to_keep)

        if state['n_iter'] % cfg.saving.log_low_freq == 0 or state['n_iter'] == cfg.training.n_iters-1:
            for logger in low_freq_loggers:
                logger(state=state, cfg=cfg, writer=writer,
                       minibatch=minibatch, dataset=dataset)

        state['n_iter'] += 1
        if state['n_iter'] > cfg.training.n_iters - 1:
            exit_flag = True
            break

    if exit_flag:
        break

writer.close()

return save_dir

Training with config piano_test
number of parameters:  7022338


 61%|██████    | 57/94 [00:06<00:04,  8.90it/s]


SystemExit: 0

In [11]:
%debug

> /home/groups/swl1/yixiuz/torch_fid/tauLDR/lib/losses/losses.py(481)calc_loss()
    479         writer.add_scalar('reg', reg_mean.detach(), state['n_iter'])
    480 
--> 481         assert(not sig_mean.isnan().any())
    482         assert(not reg_mean.isnan().any())
    483 

ipdb> sig_mean.isnan().any()
tensor(True, device='cuda:0')
ipdb> reg_mean.isnan().any()
tensor(False, device='cuda:0')
ipdb> ll
    258     def calc_loss(self, minibatch, state, writer):
    259         model = state['model']
    260         S = self.cfg.data.S
    261         if len(minibatch.shape) == 4:
    262             B, C, H, W = minibatch.shape
    263             minibatch = minibatch.view(B, C*H*W)
    264         B, D = minibatch.shape
    265         device = model.device
    266 
    267         ts = torch.rand((B,), device=device) * (1.0 - self.min_time) + self.min_time
    268 
    269         qt0 = model.transition(ts) # (B, S, S)
    270 
    271         rate = model.rate(ts) # (B, S, S)
    2

ipdb> sig_norm 
tensor([0.9610, 0.9448, 1.0522, 1.0044, 1.1113, 0.9209, 0.7087, 1.0361, 1.3632,
        1.5989, 1.0880, 1.1363, 0.9507, 2.0568, 0.7775, 1.1533, 1.0865, 0.8273,
        0.8949, 0.9655,    nan, 0.9381, 1.0578, 0.9631, 0.8950, 0.6514, 1.0022,
        0.9261, 1.0820, 0.8319, 0.9276, 0.9749, 1.0042, 0.9525, 0.9573, 1.0258,
        0.8892, 0.9851, 1.1709, 0.7530, 1.0594, 0.9841, 1.3866, 1.2225, 0.8623,
        1.2098, 0.9155, 1.0089, 0.9285, 1.0262, 1.0443, 0.9809, 1.0980, 1.4105,
        0.9752, 1.0549, 0.8479, 1.0820, 0.6409, 0.9799, 1.2709, 0.9894, 1.0470,
        1.2403], device='cuda:0')
ipdb> outer_sum_sig
tensor([-1011.0837,  -878.2532, -1055.2888,  -625.3335, -1342.3048,  -502.0121,
         -571.7645,  -829.2614,  -540.4450, -1665.6967, -1888.5383,  -851.0900,
        -1460.8113,  -450.3931,  -921.6508, -1440.2085, -1482.6438,  -648.2577,
         -635.0673,  -968.4739, -2014.8630, -1086.9919, -1104.7473,  -687.7708,
        -1341.5536, -1890.3142,  -612.4836, -1153.

ipdb> qt0_sig_norm_numer[20]
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.9830],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.9830],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.9830],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.9830],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.9830],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.9830]],
       device='cuda:0')
ipdb> (Z_sig_norm * qt0_sig_norm_denom.view(B,d,1))
tensor([[[ 95.6386,  95.6386,  95.6386,  ...,  95.6386,  95.6386,  94.9196],
         [131.8680, 131.8680, 131.8680,  ..., 131.8680, 131.8680, 130.8690],
         [131.8680, 131.8680, 131.8680,  ..., 131.8680, 131.8680, 130.8690],
         ...,
         [ 95.6386,  95.6386,  95.6386,  ...,  95.6386,  95.6386,  94.9196],
         [131.8680, 131.8680, 131.8680,  ..., 131.8680, 131.8680, 130.8690],
         [131.8680, 131.8680, 131.8680,  ..., 131.8680, 131.8680, 130.8690]],

        [[ 71.9004,

ipdb> (rate_sig_norm * qt0_sig_norm_numer * x_tilde_mask)[20]]
*** SyntaxError: unmatched ']'
ipdb>  (rate_sig_norm * qt0_sig_norm_numer * x_tilde_mask)[20]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
ipdb>  (rate_sig_norm * qt0_sig_norm_numer * x_tilde_mask)[20].shape
torch.Size([224, 130])
ipdb> ((Z_sig_norm * qt0_sig_norm_denom.view(B,d,1))[20]==0)
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]], device='cuda:0')
ipdb> ((Z_sig_norm * qt0_sig_norm_denom.

ipdb> Z_sig_norm[20,0]
tensor([117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563, 117.6563,
        117.6563, 117.6563, 117.6563, 117

In [43]:
cfg.data

S: 130
batch_size: 64
name: LakhPianoroll
path: /home/groups/swl1/yixiuz/torch_fid/downloads/pianoroll_dataset/train.npy
shape:
- 256
shuffle: true